In [ ]:
import numpy as np
import pandas as pd
import datetime
import math
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/storage/opt_project_test/optProjectTest/optProjectTest/data/build_campaign_model.csv' )

In [ ]:
init_bid = [10, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12, 12]
bid =      [10, 10, 11, 11, 13, 14, 13, 15, 15, 14, 16, 16]
daily_charge=np.full((12,), 322)
daily_budget=np.full((12,), 1650)
df_bid = pd.DataFrame({'init_bid':init_bid, 'bid':bid, 'daily_charge':daily_charge, 'daily_budget':daily_budget})
df_bid

In [ ]:
df_camp = pd.concat( [df, df_bid], axis=1 )
df_camp

In [ ]:
df_train_x = df_camp[['init_bid', 'Impressions', 'Amount Spent (TWD)', 'Clicks (All)', 'CPC (All) (TWD)', 'daily_budget', 'daily_charge']]
df_train_y = df_camp[['Impressions','Amount Spent (TWD)', 'Clicks (All)', 'CPC (All) (TWD)', 'bid']]

In [ ]:
df_train_x = df_train_x.drop(df_train_x.index[len(df_train_x)-1])


In [ ]:
df_train_y_shift = df_train_y.shift(-1).dropna()

In [ ]:
df_train_y_shift

In [ ]:
def scaleData(train_X , train_Y):
    train_X_arr = train_X.values
    train_Y_arr = train_Y.values
    
    scalerX = StandardScaler().fit(train_X_arr)
    scaled_X = scalerX.transform(train_X_arr)
    scalerY = StandardScaler().fit(train_Y_arr)
    scaled_Y = scalerY.transform(train_Y_arr)
    return scalerX, scaled_X, scalerY, scaled_Y
scalerX, scaled_X, scalerY, scaled_Y = scaleData(df_train_x, df_train_y_shift)

In [ ]:
scalerX

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import optimizers, metrics
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.externals import joblib 
BATCH_SIZE = 1
ITERATION = 10
EPOCH = 100
NEURONS = 32
MODEL_NAME = 'RL_Env_' +  str(ITERATION) + '_' + str(EPOCH) + '_' + str(NEURONS) + '.h5'

In [ ]:
def create_model(train_x, train_y, batch_size, iteration, neurons, epoch):
    x, y = train_x, train_y
    x = train_x.reshape(train_x.shape[0], 1, train_x.shape[1])
    
    model = Sequential()
    model.add(LSTM( int(neurons) , batch_input_shape=(batch_size, x.shape[1], x.shape[2]), stateful = False, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(neurons, batch_input_shape=(batch_size, x.shape[1], x.shape[2]), stateful=False))
    model.add(Dropout(0.2))
    model.add(Dense(train_y.shape[1]))
    model.compile(loss='mse', optimizer='adam',metrics=[metrics.mse])
    
    es = EarlyStopping(monitor='val_loss', patience=10)
    history = model.fit(x, y, epochs=EPOCH, batch_size=batch_size, verbose=1, shuffle=False, validation_split=0.2, callbacks=[es])
    plt.plot(history.history['val_loss'], 'r')
    
#     for i in range(iteration):
#         print('====round', str(i) +  '/' + str(iteration))
#         history = model.fit(x, y, epochs=EPOCH, batch_size=batch_size,
#                   verbose=1, shuffle=False, validation_split=0.2, callbacks=[es])
#         #plt.plot(history.history['val_loss'], 'r')
#         model.reset_states()
    model.save(MODEL_NAME)
    return model, history

In [ ]:
%%time
model, history = create_model(scaled_X, scaled_Y, BATCH_SIZE, ITERATION, NEURONS, EPOCH)

In [ ]:
MODEL_NAME = '/storage/opt_project_test/optProjectTest/optProjectTest/codes/RL_Env_10_100_32.h5'

In [ ]:
def load_LSTM_model(MODEL_NAME):
    global model
    model = load_model(MODEL_NAME)
    return model
def evaluateModel(test_X, scalerX):
    test_X = scalerX.transform(test_X)
    test_X = test_X.reshape(test_X.shape[0],1,test_X.shape[1])
#     for i,arr in enumerate(test_X):
    ans = model.predict(test_X[0:1,:,:])
    ans = scalerY.inverse_transform(ans)
    prediction = pd.DataFrame(data=ans, columns=['Impressions','Amount Spent (TWD)', 'Clicks (All)', 'CPC (All) (TWD)', 'bid'])
    return prediction

In [ ]:
load_LSTM_model(MODEL_NAME)
df_next_state = evaluateModel(df_train_x, scalerX)

In [ ]:
df_next_state

In [ ]:
df_train_y_shift.head(2)

In [ ]:
df_terminal_state = pd.DataFrame(data=np.array([[10, 50000, 10000, 5, 1650, 322]]),
                     columns=['init_bid', 'Amount Spent (TWD)', 'Clicks (All)','CPC (All) (TWD)','daily_budget','daily_charge'])


In [ ]:
df_terminal_state.info()

In [ ]:
final_days = 31
campaign_days = 31
least_days = 25
w_kpi = 1
w_spend = 0.5 
w_bid = 0.5
w_time = 1
w_init_bid=1
m_kpi = df_terminal_state['Clicks (All)']/df_terminal_state['daily_charge']
m_spend = ( df_terminal_state['daily_budget']-df_terminal_state['Amount Spent (TWD)'] ) / df_terminal_state['daily_budget']
m_bid = ( df_terminal_state['CPC (All) (TWD)']-df_terminal_state['CPC (All) (TWD)'] ) / df_terminal_state['CPC (All) (TWD)']
m_time = final_days / campaign_days
m_init_bid = (df_terminal_state['init_bid']-df_terminal_state['CPC (All) (TWD)'])/df_terminal_state['CPC (All) (TWD)']
q_terminal = w_kpi*m_kpi-w_spend*m_spend+w_bid*m_bid-w_init_bid*m_init_bid
R = 1
r = 0.1
q_terminal

In [ ]:
q_target_day = q_terminal / campaign_days
q_least_day = q_terminal / least_days
q_target_day, q_least_day

In [ ]:
def terminal_state():
    df_terminal_state = pd.DataFrame(data=np.array([[10, 50000, 10000, 5, 1650, 322]]),
                         columns=['init_bid', 'Amount Spent (TWD)', 'Clicks (All)','CPC (All) (TWD)','daily_budget','daily_charge'])
    return df_terminal_state, 31

In [ ]:
def init_state():
    df_init_state = pd.DataFrame(data=np.array([[10, 0, 0, 0, 20, 1650, 322]]),
                             columns=['init_bid', 'Impressions', 'Amount Spent (TWD)', 'Clicks (All)', 'CPC (All) (TWD)', 'daily_budget', 'daily_charge'])
    return df_init_state, 0

In [ ]:
def q_state_compute(df, day):
    m_kpi = df['Clicks (All)'] / df['daily_charge']
    m_spend = (df_terminal_state['daily_budget']-df['Amount Spent (TWD)'])/df_terminal_state['daily_budget'] 
    m_bid = (df_terminal_state['CPC (All) (TWD)']-df['CPC (All) (TWD)'])/df_terminal_state['CPC (All) (TWD)']
    m_init_bid = (df['init_bid']-df_terminal_state['CPC (All) (TWD)'])/df_terminal_state['CPC (All) (TWD)']
#     m_time = day / campaign_days
    q_state = w_kpi*m_kpi-w_spend*m_spend+w_bid*m_bid-w_init_bid*m_init_bid#+w_time*m_time
    return q_state

In [ ]:
def predict_next_state(df):
    df_next_state = evaluateModel(df, scalerX)
    return df_next_state

In [ ]:
df_terminal_state, day = terminal_state()
q_terminal = q_state_compute(df_terminal_state, day)

In [ ]:
init_bid=10
daily_budget=1650
daily_charge=322
def next_state_conversion(df, init_bid, daily_budget, daily_charge):
    df_temp = pd.DataFrame(data=np.array([[init_bid, daily_budget, daily_charge]]), columns=['init_bid', 'daily_budget', 'daily_charge'])
    df_next_state = pd.concat([df, df_temp], axis=1)
    return df_next_state

In [ ]:
df_init_state, day = init_state()
df_terminal_state, day = terminal_state()
q_init_state = q_state_compute(df_init_state, day)
q_terminal_state = q_state_compute(df_terminal_state, day)

In [ ]:
df_next_state = predict_next_state(df_init_state)
df_next_state = next_state_conversion(df_next_state, init_bid, daily_budget, daily_charge)
q_next_state = q_state_compute(df_next_state, 0)
q_next_state

In [ ]:
import numpy as np
gamma = 0.8
epsilon = 0.5
alpha = 0.9
days=0
init_bid=10
df_init_state, day = init_state()
df_terminal_state, day = terminal_state()
q_init_state = q_state_compute(df_init_state, day)
q_teminal_state = q_state_compute(df_terminal_state, day)
root = TreeNode(q_init_state)
mytreenode=root

In [ ]:
df_state = df_init_state
q_state = q_init_state



ACTION = [init_bid-1, init_bid+0, init_bid+1]
for i, action in enumerate(ACTION):
    INIT_BID = action
    df_state_prime = predict_next_state(df_init_state)
    
    df_state_prime = next_state_conversion(df_state_prime, INIT_BID, daily_budget, daily_charge)
    q_state_prime = q_state_compute(df_state_prime, day)
    if i == 0:
        currentnode = root.insertLeft(q_state_prime)
    elif i == 1:
        currentnode = root.insertMid(q_state_prime)
    elif i == 2:
        currentnode = root.insertRight(q_state_prime)
    
#         1.bid調整步伐大一點
#         2.時間間隔細一點
    time_reward = -0.1
    reward = -0.1
    print(q_target_day.iloc[0], q_least_day.iloc[0], q_state_prime.iloc[0], action)
    if q_state_prime.iloc[0] > q_target_day.iloc[0] and q_state_prime.iloc[0] < q_least_day.iloc[0]:#判斷決定是否正巷或負向reward
        time_reward = 0.1
        reward = 0.1
#         possible_actions.append(action)
#         possible_q.append(q_next_state) 

### Update Q value
q_state = q_state + alpha * ( reward+reward_time + gamma * q_state_prime.max() - q_state )
currentnode.replaceSelfData(q_state)
#             [[q1,a1], s01  s0
#              [q2,a2], s02  s0
#              [q3,a3]  s03  s0]
#     days+1
    
#     if np.random.random() < epsilon:
#         # choose random action
#         action = possible_actions[np.random.randint(0, len(possible_actions))]
#     else:
#         # greedy
#         action = possible_actions[np.argmax(possible_q)]
        
        
#         q_next_state = q_state_compute(df_current_state, 0)

In [53]:
class TreeNode(object):
    def __init__(self,val,parent=None):
        self.val=val
        self.left=None
        self.mid=None
        self.right=None
        self.parent=parent
        self.currentnode=self
    def insertLeft(self,val,parent=None,currentnode=None):
        if self.left == None:
            self.left = TreeNode(val, parent=self)
            self.currentnode = self.left
#             print(self.left)
#             print(self.currentnode)
#             print(self.left.val)
#             print(self.currentnode.val)
            return self.currentnode
        node_index = self.left
        self.left.insertLeft(val,currentnode=node_index)
    
    def insertMid(self,val,parent=None):
        if self.mid == None:
            self.mid = TreeNode(val, parent=parent)
            self.mid.parent = self
            currentnode = self.left
            return currentnode
        else:
            self.mid.insertMid(val)
    
    def insertRight(self,val,parent=None):
        currentnode = self
        if self.right == None:
            self.right = TreeNode(val, parent=parent)
            self.right.parent = self
            currentnode = self.left
            return currentnode
        else:
            self.right.insertRight(val)
    
    def replaceSelfData(self, val):
        self.val=val
        return self.val
    def replaceNodeData(self, val):
        self.val = val
        return self.val
    def getAllChildData(self):
        result=dict()
        if self.left != None and self.mid != None and self.right != None:
            result[0] = self.left.val
            result[1] = self.mid.val
            result[2] = self.right.val
        else:
            raise KeyError('Error, current node has no child.')
        return result
    def getMaxChildData(self):
        if self.left != None and self.mid != None and self.right != None:
            result = self.getAllChildData()
            print(result)
            key = sorted(result, key=result.get, reverse=True)
            print(result.pop(key[1], key[2]))
            

In [54]:
mytreenode = TreeNode(0.0)

In [ ]:
mytreenode.left

In [ ]:
currentnode = mytreenode.insertLeft(100)

In [ ]:
currentnode.currentnode.val

In [ ]:
node = mytreenode.insertLeft(200)

In [ ]:
node

In [ ]:
currentnode = mytreenode.insertLeft(300)

In [ ]:
mytreenode.left.left.left.val

In [55]:
mytreenode.insertLeft(200)
mytreenode.insertMid(201)
mytreenode.insertRight(202)

In [56]:
mytreenode.getMaxChildData()

{0: 200, 1: 201, 2: 202}
201


In [ ]:
currentnode.left.left.parent.val